In [ ]:
import numpy as np
import optuna
import matplotlib.pyplot as plt
from bcmix import *

In [ ]:
ACTION_RANGE = (-5.0, 5.0)
N_TRIALS = 100
DATA_LEN = 80
P = 0.04

In [ ]:
# prior
mean_0 = np.array([[0.0], [0.0]])
covm_0 = np.array([[2.0, 0.0], [0.0, 2.0]])

canonical_0 = np.array([[0.0], [0.0]])
precision_0 = np.array([[1.0, 0.0], [0.0, 1.0]])
canonicals_0 = [canonical_0]
precisions_0 = [precision_0]
logcons_0 = [np.linalg.slogdet(precision_0)[1] / 2 - (canonical_0.T @ np.linalg.inv(precision_0) @ canonical_0).item() / 2]
pit_0 = [0]

# true value
alpha, beta = 1.8, -2.4
print(alpha, beta, myopic(canonical_0, precision_0))

### Rollout with BCMIX

In [ ]:
def objective_bcmix(trial):
    a = trial.suggest_float('a', ACTION_RANGE[0], ACTION_RANGE[1])
    q = q_myopic_with_change(canonicals_i, precisions_i, logcons_i, pit_i, a, alpha_i, beta_i, mean_0, covm_0, p=P)
    return q

In [ ]:
for i in range(1):
    # initialize
    canonicals_i, precisions_i, logcons_i, pit_i = canonicals_0.copy(), precisions_0.copy(), logcons_0.copy(), pit_0.copy()
    alpha_i, beta_i = alpha, beta
    for j in range(DATA_LEN):
        # current state
        # select action
        #study = optuna.create_study(direction="maximize")
        #study.optimize(objective_bcmix, n_trials=N_TRIALS)
        #a = study.best_trial.params['a']
        a = np.random.uniform(ACTION_RANGE[0], ACTION_RANGE[1])
        # update state
        y, alpha_i, beta_i = env_response(a, alpha_i, beta_i, mean_0, covm_0, p=P)
        canonicals_i, precisions_i, logcons_i, pit_i, loglike = update_with_change(canonicals_i, precisions_i, logcons_i, pit_i, a, y, p=P)
        print(alpha_i, beta_i)
        print(pit_i)


In [ ]:
def objective(trial):
    p = trial.suggest_float('p', 0.0, 0.1)
    loglike = marginal_likelihood(p, canonicals_0, precisions_0, logcons_0, pit_0, xs, ys)
    return loglike